In [85]:
# Import necessary packages. 
from popstar import synthetic, evolution, atmospheres, reddening, ifmr
from popstar.imf import imf, multiplicity
import numpy as np
import pylab as py
import pdb
import os
import pylab as py
import astropy.modeling
from astropy.table import Table, Column, MaskedColumn

In [46]:
class MultiplicityResolvedDK(multiplicity.MultiplicityUnresolved):
    """
    Sub-class of MultiplicityUnresolved that adds semimajor axis and eccentricity information for multiple objects
    """
    def __init__(self, a_amp = 379.79953034, a_break = 4.90441533, a_slope1 = -1.80171539, 
                 a_slope2 = 4.23325571, a_std_slope = 0.83984154, a_std_intercept = 0.30922467, **kwargs):
        super(MultiplicityResolvedDK, self).__init__(**kwargs)
        self.a_amp = a_amp
        self.a_break = a_break
        self.a_slope1 = a_slope1
        self.a_slope2 = a_slope2
        self.a_std_slope = a_std_slope
        self.a_std_intercept = a_std_intercept
    
    def log_semimajoraxis(self, mass):
        """
        Generate the semimajor axis for a given mass. The mean and standard deviation of a given mass are determined 
        by fitting the data from fitting the semimajor axis data as a function of mass in table 1 of Deushane and Krauss 2013.
        Then a random semimajor axis is drawn from a log normal distribution with that mean and standard deviation.
        """
        a_mean_func = astropy.modeling.powerlaws.BrokenPowerLaw1D(amplitude=self.a_amp, x_break=self.a_break, alpha_1=self.a_slope1, alpha_2=self.a_slope2)
        a_mean = np.log10(a_mean_func(mass)) #mean log(a)
        a_std_func = astropy.modeling.models.Linear1D(slope=self.a_std_slope, intercept=self.a_std_intercept)
        a_std = a_std_func(mass) #sigma_log(a)
        
        #The following are the definitions of mean and std for the lognormal as a function of the mean and std of the normal dist
        lognorm_a_mean = np.log(a_mean**2/np.sqrt(a_mean**2 + a_std**2))
        lognorm_a_std = np.sqrt(np.log(1+ a_std**2/a_mean**2))
        log_semimajoraxis = np.random.lognormal(lognorm_a_mean, lognorm_a_std)
        return log_semimajoraxis

In [90]:
class ResolvedCluster_ResolvedMult(synthetic.ResolvedCluster):
    """
    Sub-class of ResolvedCluster that adds semimajor axis info to the companions table
    """
    
    def __init__(self, iso, imf, cluster_mass,
                 ifmr=None, verbose=False, seed=None):

        synthetic.ResolvedCluster.__init__(self, iso, imf, cluster_mass, ifmr=ifmr, verbose=verbose,
                                     seed=seed)        
        if self.imf.make_multiples:
            N_comp_tot = self.star_systems['N_companions'].sum()
            
            self.companions.add_column( Column(np.zeros(N_comp_tot, dtype=float), name='log_a') )
        
            for ii in range(len(self.companions)):
                self.companions['log_a'][ii] = self.imf._multi_props.log_semimajoraxis(self.star_systems['mass'][self.companions['system_idx'][ii]])
        return
            

In [61]:
# Fetch isochrone
logAge = 6.70 # Age in log(years)
AKs = 1.0 # Ks filter extinction in mags
dist = 4000 # distance in parsecs
metallicity = 0 # metallicity in [M/H]
atm_func = atmospheres.get_merged_atmosphere
evo_merged = evolution.MISTv1()
redlaw = reddening.RedLawCardelli(3.1) # Rv = 3.1
filt_list = ['nirc2,J', 'nirc2,Kp']

iso_dir = 'iso_merged_r1/'

if not os.path.exists(iso_dir):
        os.mkdir(iso_dir)

iso_merged = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_merged, atm_func=atm_func,
                                 filters=filt_list, red_law=redlaw,
                                 iso_dir=iso_dir, mass_sampling=3)

In [79]:
# Now we can make the cluster. 
clust_mtot = 10**3.
clust_multiplicity = MultiplicityResolvedDK()

# Multiplicity is defined in the IMF object
clust_imf_Mult = imf.Kroupa_2001(multiplicity=clust_multiplicity)

In [91]:
# Make clusters
clust_Mult = ResolvedCluster_ResolvedMult(iso_merged, clust_imf_Mult, clust_mtot)

clust_Mult_ss = clust_Mult.star_systems

In [93]:
clust_Mult_ss

mass,isMultiple,systemMass,Teff,L,logg,isWR,mass_current,phase,m_nirc2_J,m_nirc2_Kp,N_companions
float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
0.6557696258545461,True,0.8893025916472839,3858.2956392523233,1.1144237162859726e+26,4.09100315927723,0.0,0.6557687154898467,-1.0,19.62067275016491,17.351697573471366,1
0.7824912742051913,False,0.7824912742051913,4010.7439938519956,1.4852138447268071e+26,4.110388521354305,0.0,0.7824901467475576,-1.0,19.62648496512316,17.353770949947002,0
0.3739725065076382,True,0.3911397667548108,3507.609546962667,4.953394821572928e+25,4.0335379149801955,0.0,0.3739720462833696,-1.0,20.624504620502123,18.36653532722497,1
0.20123710189863236,True,0.38996933177760196,3220.927989376646,2.0331855851891858e+25,4.003332932241222,0.0,0.20123688492937977,-1.0,21.451854826177396,19.28922148526161,2
0.11687188884615318,False,0.11687188884615318,3031.734801077475,9.713529866059517e+24,3.9754720919852637,0.0,0.1168717776575894,-1.0,22.16691002954095,20.049946037109308,0
0.44164484080622557,False,0.44164484080622557,3602.988520672156,6.28803780127255e+25,4.048657980861444,0.0,0.4416442772348918,-1.0,20.40664949850092,18.134096732999346,0
0.4960171672266016,True,0.9618991479767797,3672.249508004216,7.430217677080813e+25,4.060716118742023,0.0,0.49601651755464976,-1.0,19.547576134030834,17.268721744683717,1
0.3883603037621908,False,0.3883603037621908,3528.4401294801787,5.234941605709963e+25,4.036835696619693,0.0,0.38835982165348887,-1.0,20.577178678707238,18.31565413561602,0
0.4363144981000246,False,0.4363144981000246,3595.531038721608,6.182690848172327e+25,4.04747533332288,0.0,0.43631394267784196,-1.0,20.42370846198888,18.15225890969227,0


In [92]:
clust_Mult.companions

system_idx,mass,Teff,L,logg,isWR,mass_current,phase,m_nirc2_J,m_nirc2_Kp,log_a
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0,0.23353296579273775,3274.6049017146765,2.4871024291494666e+25,4.009925591677168,0.0,0.23353270586221056,-1.0,21.2599971438805,19.074871189984705,0.9170966658530465
2,0.01716726024717264,nan,nan,nan,nan,nan,nan,nan,nan,0.6505114574146387
3,0.09716110994641776,nan,nan,nan,nan,nan,nan,nan,nan,0.007557930301736444
3,0.09157111993255185,nan,nan,nan,nan,nan,nan,nan,nan,0.003650893146091805
6,0.46588198075017806,3634.207196863892,6.793757075152018e+25,4.054033323811614,0.0,0.46588137894903553,-1.0,20.340417108238658,18.06393495816988,2.310832308210212
11,0.5710574996834952,3761.9777480249395,9.107404798374258e+25,4.075375205888649,0.0,0.5710567293240439,-1.0,20.07859996642312,17.787824178735466,0.40382890650523984
12,0.3374398637643659,3447.505205226059,4.267396997061992e+25,4.024078634596665,0.0,0.33743945795602187,-1.0,20.757838276260138,18.514836073886205,0.4771152111248648
13,0.13486641962831425,3083.279734177188,1.1731130715860102e+25,3.9891114240105416,0.0,0.13486628656299568,-1.0,21.99390795783919,19.864599760970382,0.01187396361673475
14,1.056643057133348,4396.371480885272,2.6852552875304078e+26,4.1439677304628635,0.0,1.0566414294075421,-1.0,19.08774252510609,16.93604686018419,0.44487407994220474
